# Battle of Neighbors 

## Introduction and Data
Opening a New Shopping Mall in Bangalore, India

    Build a dataframe of neighborhoods in Bangalore, India by web scraping the data from Wikipedia page
    Get the geographical coordinates of the neighborhoods
    Obtain the venue data for the neighborhoods from Foursquare API
    Explore and cluster the neighborhoods
    Select the best cluster to open a new shopping mall


## Business Problem
This project is mainly focused on geospatial analysis of the Bangalore City to understand which would be the best place to open a new mall

### Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
!pip install geocoder
import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
!pip install folium
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 102kB 2.3MB/s 
Libraries imported.


### Scrap data from Wikipedia page into a DataFrame

In [0]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore").text

In [0]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [0]:
# create a list to store neighborhood data
neighborhoodList = []

In [0]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

neighborhoodList.remove("List of areas in Bangalore Cantonment")
neighborhoodList.remove("List of areas in Bengaluru Pete")
neighborhoodList.remove("List of neighbourhoods in Bangalore")

In [6]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})
kl_df.head()

,Neighborhood
0,Adugodi
1,"Agara, Bangalore"
2,Ananthnagar
3,Anjanapura
4,Arekere


In [7]:
# print the number of rows of the dataframe
kl_df.shape

(128, 1)

### Get the geographical coordinates

In [0]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bangalore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [0]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [0]:
coords

[[12.944020000000023, 77.60800000000006],
 [12.842830000000049, 77.48759000000007],
 [12.954080000000033, 77.54135000000008],
 [12.858110000000067, 77.55909000000008],
 [12.885680000000036, 77.59668000000005],
 [12.963480000000061, 77.61297000000008],
 [13.02753000000007, 77.65049000000005],
 [13.07728748449913, 77.65760577036913],
 [13.044710000000066, 77.55008000000004],
 [12.922310000000039, 77.56988000000007],
 [13.019643510687336, 77.65469211693214],
 [12.938980000000072, 77.57137000000006],
 [12.992220000000032, 77.53444000000007],
 [12.927350000000047, 77.67185000000006],
 [13.083010000000058, 77.54779000000008],
 [12.900090000000034, 77.60433000000006],
 [12.903080000000045, 77.62444000000005],
 [12.817530000000033, 77.67879000000005],
 [12.993330000000071, 77.66123000000005],
 [12.91488000000004, 77.61004000000008],
 [12.951940000000036, 77.54445000000004],
 [12.913110000000074, 77.71565000000004],
 [12.959700296061198, 77.57131179631511],
 [12.794010000000071, 77.700150000000

In [0]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [0]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [12]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df.head(10)

(128, 3)


,Neighborhood,Latitude,Longitude
0,Adugodi,12.944020,77.608000
1,"Agara, Bangalore",12.842830,77.487590
2,Ananthnagar,12.954080,77.541350
3,Anjanapura,12.858110,77.559090
4,Arekere,12.885680,77.596680
5,Austin Town,12.963480,77.612970
6,Babusapalya,13.027530,77.650490
7,"Bagalur, Bangalore Urban",13.077287,77.657606
8,Bahubalinagar,13.044710,77.550080
9,Banashankari,12.922310,77.569880


In [0]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

###  Create a map of Bangalore with neighborhoods superimposed on top

In [13]:

address = 'Bangalore, Karnataka, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore, India 12.9791198, 77.5912997.


In [14]:
# create map of Bangalore using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [0]:
# save the map as HTML file
map_kl.save('map_kl.html')

### Use the Foursquare API to explore the neighborhoods 

In [0]:
# define Foursquare Credentials and Version
# @hidden_cell
CLIENT_ID = '3Z3LMLWE5LMALWTAOHL50BJNY1W3QNRQTXCTCIYWEG0BDFXW' # Put Your Client Id
CLIENT_SECRET = 'IUYJOZKBMBLWE3MIKIMRZ2TZCC1V2EOQ5T4LDAPJIVSDCPLC' # Put You Client Secret 
VERSION = '20180604'

### Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [0]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [18]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6667, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adugodi,12.94402,77.608,Tommy Hilfiger,12.934552,77.611347,Clothing Store
1,Adugodi,12.94402,77.608,PVR IMAX,12.934595,77.611321,Movie Theater
2,Adugodi,12.94402,77.608,Lot Like Crêpes,12.936421,77.613284,Creperie
3,Adugodi,12.94402,77.608,Truffles Ice & Spice,12.933443,77.614265,Burger Joint
4,Adugodi,12.94402,77.608,Zingron - Naga Kitchen,12.936271,77.615051,Indian Restaurant
